SKRIPSI

# Import library

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
import nltk

from nltk import ngrams
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, validation_curve,RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from matplotlib_venn import venn2
from time import time

np.random.seed(0)
stopwords = nltk.corpus.stopwords.words('english')
poster_Stemmer = nltk.PorterStemmer()


# Baca file csv

In [2]:
#baca csv
data_train_amazon = pd.read_csv('Amazon_Train.csv')
data_train_yelp = pd.read_csv('Yelp_Train.csv')
data_test_amazon = pd.read_csv('Amazon_Test.csv')
data_test_yelp = pd.read_csv('Yelp_Test.csv')

In [3]:
print(data_train_amazon.head())
print("\n")
print(data_train_yelp.head())

   Label                                           Sentimen
0      0  Buyer beware This is a self-published book, an...
1      0  The Worst! A complete waste of time. Typograph...
2      0  Oh please I guess you have to be a romance nov...
3      0  Awful beyond belief! I feel I have to write to...
4      0  Another Abysmal Digital Copy Rather than scrat...


   Label                                           Sentimen
0      0  I don't know what Dr. Goldberg was like before...
1      0  I'm writing this review to give you a heads up...
2      0  Owning a driving range inside the city limits ...
3      0  This place is absolute garbage...  Half of the...
4      0  Used to go there for tires, brakes, etc.  Thei...


In [4]:
print(data_test_amazon.head())
print("\n")
print(data_test_yelp.head())

   Label                                           Sentimen
0      0  Overly complicated Being both a U.S. history b...
1      0  Terrible Disappointment -- parts don't fit My ...
2      0  Didn't hold up.....very disappointed I bought ...
3      0  gene hates jezebel i love JLJ but this compila...
4      0  Nice toy but ... My six-year-old loves space a...


   Label                                           Sentimen
0      0  My wife and I used to love Arriba's, til recen...
1      0  You get what you pay for.  The food is inexpen...
2      0  Unfortunately, yesterday's visit was one of th...
3      0  I went into the Scottsdale location yesterday....
4      0  It takes a lot for me to write a review and bl...


# Preprocessing ke lowercase

In [5]:
iterator = 0
data_Preprocessing_Amazon_Train = data_train_amazon
data_Lowercase_Amazon_Train = []

data_Preprocessing_Yelp_Train = data_train_yelp
data_Lowercase_Yelp_Train = []

while iterator < len(data_train_amazon) :
    data_Lowercase_Amazon_Train.append(data_train_amazon.Sentimen[iterator].lower())
    iterator = iterator + 1

iterator = 0

while iterator < len(data_train_yelp) :
    data_Lowercase_Yelp_Train.append(data_train_yelp.Sentimen[iterator].lower())
    iterator = iterator + 1
    

In [6]:
iterator = 0
data_Preprocessing_Amazon_Test = data_test_amazon
data_Lowercase_Amazon_Test = []

data_Preprocessing_Yelp_Test = data_test_yelp
data_Lowercase_Yelp_Test = []

while iterator < len(data_test_amazon) :
    data_Lowercase_Amazon_Test.append(data_test_amazon.Sentimen[iterator].lower())
    iterator = iterator + 1

iterator = 0

while iterator < len(data_test_yelp) :
    data_Lowercase_Yelp_Test.append(data_test_yelp.Sentimen[iterator].lower())
    iterator = iterator + 1

In [7]:
data_Preprocessing_Amazon_Train['Lowercase'] = data_Lowercase_Amazon_Train
data_Preprocessing_Amazon_Train.head()

,Label,Sentimen,Lowercase
0,0,"Buyer beware This is a self-published book, an...","buyer beware this is a self-published book, an..."
1,0,The Worst! A complete waste of time. Typograph...,the worst! a complete waste of time. typograph...
2,0,Oh please I guess you have to be a romance nov...,oh please i guess you have to be a romance nov...
3,0,Awful beyond belief! I feel I have to write to...,awful beyond belief! i feel i have to write to...
4,0,Another Abysmal Digital Copy Rather than scrat...,another abysmal digital copy rather than scrat...


In [8]:
data_Preprocessing_Amazon_Test['Lowercase'] = data_Lowercase_Amazon_Test
data_Preprocessing_Amazon_Test.head()

,Label,Sentimen,Lowercase
0,0,Overly complicated Being both a U.S. history b...,overly complicated being both a u.s. history b...
1,0,Terrible Disappointment -- parts don't fit My ...,terrible disappointment -- parts don't fit my ...
2,0,Didn't hold up.....very disappointed I bought ...,didn't hold up.....very disappointed i bought ...
3,0,gene hates jezebel i love JLJ but this compila...,gene hates jezebel i love jlj but this compila...
4,0,Nice toy but ... My six-year-old loves space a...,nice toy but ... my six-year-old loves space a...


In [9]:
data_Preprocessing_Yelp_Train['Lowercase'] = data_Lowercase_Yelp_Train
data_Preprocessing_Yelp_Train.head()

,Label,Sentimen,Lowercase
0,0,I don't know what Dr. Goldberg was like before...,i don't know what dr. goldberg was like before...
1,0,I'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...
2,0,Owning a driving range inside the city limits ...,owning a driving range inside the city limits ...
3,0,This place is absolute garbage... Half of the...,this place is absolute garbage... half of the...
4,0,"Used to go there for tires, brakes, etc. Thei...","used to go there for tires, brakes, etc. thei..."


In [10]:
data_Preprocessing_Yelp_Test['Lowercase'] = data_Lowercase_Yelp_Test
data_Preprocessing_Yelp_Test.head()

,Label,Sentimen,Lowercase
0,0,"My wife and I used to love Arriba's, til recen...","my wife and i used to love arriba's, til recen..."
1,0,You get what you pay for. The food is inexpen...,you get what you pay for. the food is inexpen...
2,0,"Unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th..."
3,0,I went into the Scottsdale location yesterday....,i went into the scottsdale location yesterday....
4,0,It takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...


# Menghilangkan angka

In [11]:
iterator = 0
data_RemoveNumber_Amazon_Train = []

data_RemoveNumber_Yelp_Train = []

while iterator < len(data_Preprocessing_Amazon_Train) :
    data_RemoveNumber_Amazon_Train.append(re.sub(r"\d+", "",data_Preprocessing_Amazon_Train.Lowercase[iterator]))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Train) :
    data_RemoveNumber_Yelp_Train.append(re.sub(r"\d+", "",data_Preprocessing_Yelp_Train.Lowercase[iterator]))
    iterator = iterator + 1

In [12]:
iterator = 0
data_RemoveNumber_Amazon_Test = []

data_RemoveNumber_Yelp_Test = []

while iterator < len(data_Preprocessing_Amazon_Test) :
    data_RemoveNumber_Amazon_Test.append(re.sub(r"\d+", "",data_Preprocessing_Amazon_Test.Lowercase[iterator]))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Test) :
    data_RemoveNumber_Yelp_Test.append(re.sub(r"\d+", "",data_Preprocessing_Yelp_Test.Lowercase[iterator]))
    iterator = iterator + 1

In [13]:
data_Preprocessing_Amazon_Train['RemoveNumber'] = data_RemoveNumber_Amazon_Train
data_Preprocessing_Amazon_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber
0,0,"Buyer beware This is a self-published book, an...","buyer beware this is a self-published book, an...","buyer beware this is a self-published book, an..."
1,0,The Worst! A complete waste of time. Typograph...,the worst! a complete waste of time. typograph...,the worst! a complete waste of time. typograph...
2,0,Oh please I guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...
3,0,Awful beyond belief! I feel I have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief! i feel i have to write to...
4,0,Another Abysmal Digital Copy Rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...


In [14]:
data_Preprocessing_Amazon_Test['RemoveNumber'] = data_RemoveNumber_Amazon_Test
data_Preprocessing_Amazon_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber
0,0,Overly complicated Being both a U.S. history b...,overly complicated being both a u.s. history b...,overly complicated being both a u.s. history b...
1,0,Terrible Disappointment -- parts don't fit My ...,terrible disappointment -- parts don't fit my ...,terrible disappointment -- parts don't fit my ...
2,0,Didn't hold up.....very disappointed I bought ...,didn't hold up.....very disappointed i bought ...,didn't hold up.....very disappointed i bought ...
3,0,gene hates jezebel i love JLJ but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...
4,0,Nice toy but ... My six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but ... my six-year-old loves space a...


In [15]:
data_Preprocessing_Yelp_Train['RemoveNumber'] = data_RemoveNumber_Yelp_Train
data_Preprocessing_Yelp_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber
0,0,I don't know what Dr. Goldberg was like before...,i don't know what dr. goldberg was like before...,i don't know what dr. goldberg was like before...
1,0,I'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...
2,0,Owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...
3,0,This place is absolute garbage... Half of the...,this place is absolute garbage... half of the...,this place is absolute garbage... half of the...
4,0,"Used to go there for tires, brakes, etc. Thei...","used to go there for tires, brakes, etc. thei...","used to go there for tires, brakes, etc. thei..."


In [16]:
data_Preprocessing_Yelp_Test['RemoveNumber'] = data_RemoveNumber_Yelp_Test
data_Preprocessing_Yelp_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber
0,0,"My wife and I used to love Arriba's, til recen...","my wife and i used to love arriba's, til recen...","my wife and i used to love arriba's, til recen..."
1,0,You get what you pay for. The food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for. the food is inexpen...
2,0,"Unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th..."
3,0,I went into the Scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday....
4,0,It takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...


# Menghilangkan tanda baca

In [17]:
iterator = 0
data_RemovePunctuation_Amazon_Train = []

data_RemovePunctuation_Yelp_Train = []

while iterator < len(data_Preprocessing_Amazon_Train) :
    data_RemovePunctuation_Amazon_Train.append(data_Preprocessing_Amazon_Train.RemoveNumber[iterator].translate(str.maketrans('','', string.punctuation)))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Train) :
    data_RemovePunctuation_Yelp_Train.append(data_Preprocessing_Yelp_Train.RemoveNumber[iterator].translate(str.maketrans('','', string.punctuation)))
    iterator = iterator + 1

In [18]:
iterator = 0
data_RemovePunctuation_Amazon_Test = []

data_RemovePunctuation_Yelp_Test = []

while iterator < len(data_Preprocessing_Amazon_Test) :
    data_RemovePunctuation_Amazon_Test.append(data_Preprocessing_Amazon_Test.RemoveNumber[iterator].translate(str.maketrans('','', string.punctuation)))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Test) :
    data_RemovePunctuation_Yelp_Test.append(data_Preprocessing_Yelp_Test.RemoveNumber[iterator].translate(str.maketrans('','', string.punctuation)))
    iterator = iterator + 1

In [19]:
data_Preprocessing_Amazon_Train['RemovePunctuation'] = data_RemovePunctuation_Amazon_Train
data_Preprocessing_Amazon_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation
0,0,"Buyer beware This is a self-published book, an...","buyer beware this is a self-published book, an...","buyer beware this is a self-published book, an...",buyer beware this is a selfpublished book and ...
1,0,The Worst! A complete waste of time. Typograph...,the worst! a complete waste of time. typograph...,the worst! a complete waste of time. typograph...,the worst a complete waste of time typographic...
2,0,Oh please I guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...
3,0,Awful beyond belief! I feel I have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief i feel i have to write to ...
4,0,Another Abysmal Digital Copy Rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...


In [20]:
data_Preprocessing_Amazon_Test['RemovePunctuation'] = data_RemovePunctuation_Amazon_Test
data_Preprocessing_Amazon_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation
0,0,Overly complicated Being both a U.S. history b...,overly complicated being both a u.s. history b...,overly complicated being both a u.s. history b...,overly complicated being both a us history buf...
1,0,Terrible Disappointment -- parts don't fit My ...,terrible disappointment -- parts don't fit my ...,terrible disappointment -- parts don't fit my ...,terrible disappointment parts dont fit my son...
2,0,Didn't hold up.....very disappointed I bought ...,didn't hold up.....very disappointed i bought ...,didn't hold up.....very disappointed i bought ...,didnt hold upvery disappointed i bought this o...
3,0,gene hates jezebel i love JLJ but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...
4,0,Nice toy but ... My six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but my sixyearold loves space and to...


In [21]:
data_Preprocessing_Yelp_Train['RemovePunctuation'] = data_RemovePunctuation_Yelp_Train
data_Preprocessing_Yelp_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation
0,0,I don't know what Dr. Goldberg was like before...,i don't know what dr. goldberg was like before...,i don't know what dr. goldberg was like before...,i dont know what dr goldberg was like before ...
1,0,I'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,im writing this review to give you a heads up ...
2,0,Owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...
3,0,This place is absolute garbage... Half of the...,this place is absolute garbage... half of the...,this place is absolute garbage... half of the...,this place is absolute garbage half of the te...
4,0,"Used to go there for tires, brakes, etc. Thei...","used to go there for tires, brakes, etc. thei...","used to go there for tires, brakes, etc. thei...",used to go there for tires brakes etc their p...


In [22]:
data_Preprocessing_Yelp_Test['RemovePunctuation'] = data_RemovePunctuation_Yelp_Test
data_Preprocessing_Yelp_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation
0,0,"My wife and I used to love Arriba's, til recen...","my wife and i used to love arriba's, til recen...","my wife and i used to love arriba's, til recen...",my wife and i used to love arribas til recentl...
1,0,You get what you pay for. The food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for the food is inexpens...
2,0,"Unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...",unfortunately yesterdays visit was one of the ...
3,0,I went into the Scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday ...
4,0,It takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...


# Menghilangkan Non alfabet

In [23]:
iterator = 0

data_Regex_alpabet_only_Amazon_Train = []

data_Regex_alpabet_only_Yelp_Train = []


while iterator < len(data_Preprocessing_Amazon_Train) :
    data_Regex_alpabet_only_Amazon_Train.append(" ".join(re.findall("[a-zA-Z]+", data_Preprocessing_Amazon_Train.RemovePunctuation[iterator])))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Train) :
    data_Regex_alpabet_only_Yelp_Train.append(" ".join(re.findall(r"[a-zA-Z]+", data_Preprocessing_Yelp_Train.RemovePunctuation[iterator])))
    iterator = iterator + 1



In [24]:
iterator = 0

data_Regex_alpabet_only_Amazon_Test = []

data_Regex_alpabet_only_Yelp_Test = []


while iterator < len(data_Preprocessing_Amazon_Test) :
    data_Regex_alpabet_only_Amazon_Test.append(" ".join(re.findall("[a-zA-Z]+", data_Preprocessing_Amazon_Test.RemovePunctuation[iterator])))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Test) :
    data_Regex_alpabet_only_Yelp_Test.append(" ".join(re.findall(r"[a-zA-Z]+", data_Preprocessing_Yelp_Test.RemovePunctuation[iterator])))
    iterator = iterator + 1

In [25]:
data_Preprocessing_Amazon_Train['Regex'] = data_Regex_alpabet_only_Amazon_Train
data_Preprocessing_Amazon_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation,Regex
0,0,"Buyer beware This is a self-published book, an...","buyer beware this is a self-published book, an...","buyer beware this is a self-published book, an...",buyer beware this is a selfpublished book and ...,buyer beware this is a selfpublished book and ...
1,0,The Worst! A complete waste of time. Typograph...,the worst! a complete waste of time. typograph...,the worst! a complete waste of time. typograph...,the worst a complete waste of time typographic...,the worst a complete waste of time typographic...
2,0,Oh please I guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...
3,0,Awful beyond belief! I feel I have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief i feel i have to write to ...,awful beyond belief i feel i have to write to ...
4,0,Another Abysmal Digital Copy Rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...


In [26]:
data_Preprocessing_Amazon_Test['Regex'] = data_Regex_alpabet_only_Amazon_Test
data_Preprocessing_Amazon_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation,Regex
0,0,Overly complicated Being both a U.S. history b...,overly complicated being both a u.s. history b...,overly complicated being both a u.s. history b...,overly complicated being both a us history buf...,overly complicated being both a us history buf...
1,0,Terrible Disappointment -- parts don't fit My ...,terrible disappointment -- parts don't fit my ...,terrible disappointment -- parts don't fit my ...,terrible disappointment parts dont fit my son...,terrible disappointment parts dont fit my son ...
2,0,Didn't hold up.....very disappointed I bought ...,didn't hold up.....very disappointed i bought ...,didn't hold up.....very disappointed i bought ...,didnt hold upvery disappointed i bought this o...,didnt hold upvery disappointed i bought this o...
3,0,gene hates jezebel i love JLJ but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...
4,0,Nice toy but ... My six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but my sixyearold loves space and to...,nice toy but my sixyearold loves space and to ...


In [27]:
data_Preprocessing_Yelp_Train['Regex'] = data_Regex_alpabet_only_Yelp_Train
data_Preprocessing_Yelp_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation,Regex
0,0,I don't know what Dr. Goldberg was like before...,i don't know what dr. goldberg was like before...,i don't know what dr. goldberg was like before...,i dont know what dr goldberg was like before ...,i dont know what dr goldberg was like before m...
1,0,I'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,im writing this review to give you a heads up ...,im writing this review to give you a heads up ...
2,0,Owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...
3,0,This place is absolute garbage... Half of the...,this place is absolute garbage... half of the...,this place is absolute garbage... half of the...,this place is absolute garbage half of the te...,this place is absolute garbage half of the tee...
4,0,"Used to go there for tires, brakes, etc. Thei...","used to go there for tires, brakes, etc. thei...","used to go there for tires, brakes, etc. thei...",used to go there for tires brakes etc their p...,used to go there for tires brakes etc their pr...


In [28]:
data_Preprocessing_Yelp_Test['Regex'] = data_Regex_alpabet_only_Yelp_Test
data_Preprocessing_Yelp_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation,Regex
0,0,"My wife and I used to love Arriba's, til recen...","my wife and i used to love arriba's, til recen...","my wife and i used to love arriba's, til recen...",my wife and i used to love arribas til recentl...,my wife and i used to love arribas til recentl...
1,0,You get what you pay for. The food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for the food is inexpens...,you get what you pay for the food is inexpensi...
2,0,"Unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...",unfortunately yesterdays visit was one of the ...,unfortunately yesterdays visit was one of the ...
3,0,I went into the Scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday ...,i went into the scottsdale location yesterday ...
4,0,It takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...


In [29]:
Clean_Amazon_Sentiment_Train = data_Preprocessing_Amazon_Train['Regex']
Clean_Yelp_Sentiment_Train = data_Preprocessing_Yelp_Train['Regex']
Label_Amazon_Train = data_Preprocessing_Amazon_Train['Label']
Label_Yelp_Train = data_Preprocessing_Yelp_Train['Label']

Clean_Amazon_Sentiment_Test = data_Preprocessing_Amazon_Test['Regex']
Clean_Yelp_Sentiment_Test = data_Preprocessing_Yelp_Test['Regex']
Label_Amazon_Test = data_Preprocessing_Amazon_Test['Label']
Label_Yelp_Test = data_Preprocessing_Yelp_Test['Label']

# N-Gram (bigram dan trigram)

## N-gram Amazon

In [30]:
iterator = 0
Vocabulary_Amazon_Train = []
ngram_amazon_train = []

while iterator < len(Clean_Amazon_Sentiment_Train) :
    ngram = 2
    sentence = Clean_Amazon_Sentiment_Train[iterator]
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
    temp = vocab

    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Amazon_Train) :
            Vocabulary_Amazon_Train.append(vocab[iterator2])
        iterator2 = iterator2 + 1
    
    ngram = 3
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
    temp = temp + vocab
    ngram_amazon_train.append(temp)
    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Amazon_Train) :
            Vocabulary_Amazon_Train.append(vocab[iterator2])
        iterator2 = iterator2 + 1
        
    iterator = iterator + 1
    vocab = None
    del vocab

    

In [31]:
iterator = 0
Vocabulary_Amazon_Test = []
ngram_amazon_test = []

while iterator < len(Clean_Amazon_Sentiment_Test) :
    ngram = 2
    sentence = Clean_Amazon_Sentiment_Test[iterator]
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
    temp = vocab

    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Amazon_Test) :
            Vocabulary_Amazon_Test.append(vocab[iterator2])
        iterator2 = iterator2 + 1
    
    ngram = 3
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
    temp = temp + vocab
    ngram_amazon_test.append(temp)
    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Amazon_Test) :
            Vocabulary_Amazon_Test.append(vocab[iterator2])
        iterator2 = iterator2 + 1
        
    iterator = iterator + 1
    vocab = None
    del vocab


## N-gram Yelp

In [32]:
iterator = 0
Vocabulary_Yelp_Train = []
ngram_yelp_train = []

while iterator < len(Clean_Yelp_Sentiment_Train) :
    ngram = 2
    sentence = Clean_Yelp_Sentiment_Train[iterator]
    
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
  
    temp = vocab

    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Yelp_Train) :
            Vocabulary_Yelp_Train.append(vocab[iterator2])
        iterator2 = iterator2 + 1
    
    ngram = 3
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
    temp = temp + vocab
    ngram_yelp_train.append(temp)
    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Yelp_Train) :
            Vocabulary_Yelp_Train.append(vocab[iterator2])
        iterator2 = iterator2 + 1
        
    iterator = iterator + 1
    

In [33]:
iterator = 0
Vocabulary_Yelp_Test = []
ngram_yelp_test = []

while iterator < len(Clean_Yelp_Sentiment_Test) :
    ngram = 2
    sentence = Clean_Yelp_Sentiment_Test[iterator]
    
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
  
    temp = vocab

    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Yelp_Test) :
            Vocabulary_Yelp_Test.append(vocab[iterator2])
        iterator2 = iterator2 + 1
    
    ngram = 3
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
    temp = temp + vocab
    ngram_yelp_test.append(temp)
    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Yelp_Test) :
            Vocabulary_Yelp_Test.append(vocab[iterator2])
        iterator2 = iterator2 + 1
        
    iterator = iterator + 1

# Me inisialisasi jumlah frekuensi Gram di Vocabulary

In [34]:
WordDictionaryCount_Amazon_Train = []
WordDictionaryCount_Yelp_Train = []
WordDictionaryCount_Amazon_Test = []
WordDictionaryCount_Yelp_Test = []

iterator = 0
while iterator < len(Clean_Amazon_Sentiment_Train):
    WordDictionaryCount_Amazon_Train.append(dict.fromkeys(Vocabulary_Amazon_Train , 0))
    iterator = iterator + 1
    
iterator = 0
while iterator < len(Clean_Yelp_Sentiment_Train):
    WordDictionaryCount_Yelp_Train.append(dict.fromkeys(Vocabulary_Yelp_Train , 0))
    iterator = iterator + 1
        

iterator = 0
while iterator < len(Clean_Amazon_Sentiment_Test):
    WordDictionaryCount_Amazon_Test.append(dict.fromkeys(Vocabulary_Amazon_Train , 0))
    iterator = iterator + 1
    
iterator = 0
while iterator < len(Clean_Yelp_Sentiment_Test):
    WordDictionaryCount_Yelp_Test.append(dict.fromkeys(Vocabulary_Yelp_Train , 0))
    iterator = iterator + 1

    
iterator = 0
while iterator < len(Clean_Amazon_Sentiment_Train) :
    for gram in ngram_amazon_train[iterator] :
        WordDictionaryCount_Amazon_Train[iterator][gram] += 1
    iterator = iterator + 1
    
iterator = 0
while iterator < len(Clean_Yelp_Sentiment_Train) :
    for gram in ngram_yelp_train[iterator] :
        WordDictionaryCount_Yelp_Train[iterator][gram] += 1
    iterator = iterator + 1
    
iterator = 0
while iterator < len(Clean_Amazon_Sentiment_Test) :
    for gram in ngram_amazon_test[iterator] :
        if gram not in Vocabulary_Amazon_Train :
            continue
        WordDictionaryCount_Amazon_Test[iterator][gram] += 1
    iterator = iterator + 1
    
iterator = 0
while iterator < len(Clean_Yelp_Sentiment_Test) :
    for gram in ngram_yelp_test[iterator] :
        if gram not in Vocabulary_Yelp_Train :
            continue
        WordDictionaryCount_Yelp_Test[iterator][gram] += 1
    iterator = iterator + 1

In [35]:
pd.DataFrame([WordDictionaryCount_Amazon_Train[0],WordDictionaryCount_Amazon_Train[1]])

,bu,uy,ye,er,r,b,be,ew,wa,ar,...,umy,myu,mmc,sez,fja,coz,ozy,kav,gju,ukw
0,1,1,1,11,9,13,5,3,4,7,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,2,5,1,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0


In [36]:
pd.DataFrame([WordDictionaryCount_Yelp_Train[0],WordDictionaryCount_Yelp_Train[1]])

,i,d,do,on,nt,t,k,kn,no,ow,...,nhg,hgn,nwc,sez,ffb,fbe,yec,nww,zeg,ypu
0,4,11,6,13,7,19,1,1,5,1,...,0,0,0,0,0,0,0,0,0,0
1,10,7,6,10,13,21,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


## Menghitung nilai TF pada setiap data Amazon dan Yelp

In [37]:
def computeFrequency(wordDict, ngram):
    freqword = {}
    ngramLength = len(ngram)
    for word, count in wordDict.items():
        freqword[word] = count
    return freqword

In [38]:
CountFrequency_Sentimen_Amazon_Train = []
CountFrequency_Sentimen_Yelp_Train = []
CountFrequency_Sentimen_Amazon_Test = []
CountFrequency_Sentimen_Yelp_Test = []

In [39]:
iterator = 0
while iterator < len(ngram_amazon_train):
    CountFrequency_Sentimen_Amazon_Train.append(computeFrequency(WordDictionaryCount_Amazon_Train[iterator], ngram_amazon_train[iterator]))
    iterator = iterator + 1

iterator = 0
while iterator < len(ngram_yelp_train):
    CountFrequency_Sentimen_Yelp_Train.append(computeFrequency(WordDictionaryCount_Yelp_Train[iterator], ngram_yelp_train[iterator]))
    iterator = iterator + 1
    
iterator = 0
while iterator < len(ngram_amazon_test):
    CountFrequency_Sentimen_Amazon_Test.append(computeFrequency(WordDictionaryCount_Amazon_Test[iterator], ngram_amazon_test[iterator]))
    iterator = iterator + 1
    
iterator = 0
while iterator < len(ngram_yelp_test):
    CountFrequency_Sentimen_Yelp_Test.append(computeFrequency(WordDictionaryCount_Yelp_Test[iterator], ngram_yelp_test[iterator]))
    iterator = iterator + 1

## Menghitung nilai IDF pada setiap data Amazon dan Yelp

In [40]:
def computeIDF(WordDict):
    idfDict = {}
    number_of_document_with_term_t_in_it = {}
    N = len(WordDict)
    
    idfDict = dict.fromkeys(WordDict[0].keys(), 0)
    for doc in WordDict:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        number_of_document_with_term_t_in_it[word] = float(val)
        idfDict[word] = np.log((1+N) / (1+val)) + 1
        
    return idfDict, number_of_document_with_term_t_in_it

In [41]:
IDF_Amazon, number_of_document_with_term_t_Amazon_Train = computeIDF(WordDictionaryCount_Amazon_Train)
IDF_Yelp, number_of_document_with_term_t_in_Yelp_Train = computeIDF(WordDictionaryCount_Yelp_Train)

In [42]:
# def computeTFIDF(Tf_Sentimen_Amazon, idfs):
#     tfidf = {}
#     for word, val in Tf_Sentimen_Amazon.items():
#         tfidf[word] = val*idfs[word]
#     return tfidf

def computeTFIDFnonScale(CountFrequency, idfs):
    tfidf = {}
    for word, val in CountFrequency.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [43]:
TFIDF_nonScale_Amazon_Train = []
TFIDF_nonScale_Yelp_Train = []
TFIDF_nonScale_Amazon_Test = []
TFIDF_nonScale_Yelp_Test = []

iterator = 0
while iterator < len(WordDictionaryCount_Amazon_Train):
    TFIDF_nonScale_Amazon_Train.append(computeTFIDFnonScale(CountFrequency_Sentimen_Amazon_Train[iterator], IDF_Amazon))
    iterator = iterator + 1
    
iterator = 0
while iterator < len(WordDictionaryCount_Yelp_Train):
    TFIDF_nonScale_Yelp_Train.append(computeTFIDFnonScale(CountFrequency_Sentimen_Yelp_Train[iterator], IDF_Yelp))
    iterator = iterator + 1
    
iterator = 0
while iterator < len(WordDictionaryCount_Amazon_Test):
    TFIDF_nonScale_Amazon_Test.append(computeTFIDFnonScale(CountFrequency_Sentimen_Amazon_Test[iterator], IDF_Amazon))
    iterator = iterator + 1
    
iterator = 0
while iterator < len(WordDictionaryCount_Yelp_Test):
    TFIDF_nonScale_Yelp_Test.append(computeTFIDFnonScale(CountFrequency_Sentimen_Yelp_Test[iterator], IDF_Yelp))
    iterator = iterator + 1

In [44]:
def computeTFIDF(tfidf_nonscaled) :
    tfidf_nonscaled = np.array(tfidf_nonscaled)
    tfidf_list = tfidf_nonscaled/sum(tfidf_nonscaled**2)**0.5
    return tfidf_list

In [45]:
TFIDF_Amazon_Train = []
iterator = 0
while iterator < len(WordDictionaryCount_Amazon_Train):
    tf_idf_list = list(TFIDF_nonScale_Amazon_Train[iterator].values())
    TFIDF_Amazon_Train.append(computeTFIDF(tf_idf_list))
    iterator = iterator + 1
    
TFIDF_Yelp_Train = []
iterator = 0
while iterator < len(WordDictionaryCount_Yelp_Train):
    tf_idf_list = list(TFIDF_nonScale_Yelp_Train[iterator].values())
    TFIDF_Yelp_Train.append(computeTFIDF(tf_idf_list))
    iterator = iterator + 1
    
TFIDF_Amazon_Test = []
iterator = 0
while iterator < len(WordDictionaryCount_Amazon_Test):
    tf_idf_list = list(TFIDF_nonScale_Amazon_Test[iterator].values())
    TFIDF_Amazon_Test.append(computeTFIDF(tf_idf_list))
    iterator = iterator + 1
    
TFIDF_Yelp_Test = []
iterator = 0
while iterator < len(WordDictionaryCount_Yelp_Test):
    tf_idf_list = list(TFIDF_nonScale_Yelp_Test[iterator].values())
    TFIDF_Yelp_Test.append(computeTFIDF(tf_idf_list))
    iterator = iterator + 1

In [46]:
Vectorizer_Amazon_Train = []
iterator  = 0
while iterator < len(TFIDF_Amazon_Train):
    Vectorizer_Amazon_Train.append(list(TFIDF_Amazon_Train[iterator]))
    iterator = iterator + 1
    
Vectorizer_Amazon_Test = []
iterator  = 0
while iterator < len(TFIDF_Amazon_Test):
    Vectorizer_Amazon_Test.append(list(TFIDF_Amazon_Test[iterator]))
    iterator = iterator + 1
    
Vectorizer_Yelp_Train = []
iterator  = 0
while iterator < len(TFIDF_Yelp_Train):
    Vectorizer_Yelp_Train.append(list(TFIDF_Yelp_Train[iterator]))
    iterator = iterator + 1
    
Vectorizer_Yelp_Test = []
iterator  = 0
while iterator < len(TFIDF_Yelp_Test):
    Vectorizer_Yelp_Test.append(list(TFIDF_Yelp_Test[iterator]))
    iterator = iterator + 1

In [47]:
DataFrame_Amazon_Train = pd.DataFrame(Vectorizer_Amazon_Train, columns = Vocabulary_Amazon_Train)
DataFrame_Amazon_Train = DataFrame_Amazon_Train.reindex(sorted(DataFrame_Amazon_Train.columns), axis=1)
print (DataFrame_Amazon_Train)

             a        a    aa        ab        ac        ad   ae        af  \
0     0.174884  0.079896  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
1     0.149341  0.046906  0.0  0.044125  0.052581  0.055971  0.0  0.000000   
2     0.122904  0.056149  0.0  0.026410  0.000000  0.000000  0.0  0.033775   
3     0.107078  0.041393  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
4     0.121365  0.022871  0.0  0.014344  0.017092  0.000000  0.0  0.000000   
...        ...       ...  ...       ...       ...       ...  ...       ...   
7495  0.082949  0.014887  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
7496  0.119551  0.033377  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
7497  0.139474  0.087614  0.0  0.029971  0.000000  0.000000  0.0  0.000000   
7498  0.103292  0.000000  0.0  0.000000  0.072736  0.000000  0.0  0.000000   
7499  0.040737  0.000000  0.0  0.000000  0.000000  0.000000  0.0  0.000000   

            ag   ah  ...  zzf  zzi  zzk  zzl  zzo  zzs  zzt  zz

In [48]:
DataFrame_Amazon_Test = pd.DataFrame(Vectorizer_Amazon_Test, columns = Vocabulary_Amazon_Train)
DataFrame_Amazon_Test = DataFrame_Amazon_Test.reindex(sorted(DataFrame_Amazon_Test.columns), axis=1)
print (DataFrame_Amazon_Test)

             a        a    aa        ab   ac        ad   ae        af  \
0     0.145900  0.061100  0.0  0.019159  0.0  0.048606  0.0  0.024502   
1     0.087064  0.013673  0.0  0.000000  0.0  0.000000  0.0  0.000000   
2     0.090990  0.048992  0.0  0.015362  0.0  0.000000  0.0  0.039293   
3     0.086429  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.000000   
4     0.085100  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.000000   
...        ...       ...  ...       ...  ...       ...  ...       ...   
2495  0.048423  0.015209  0.0  0.000000  0.0  0.000000  0.0  0.000000   
2496  0.051493  0.021564  0.0  0.000000  0.0  0.000000  0.0  0.000000   
2497  0.109906  0.046027  0.0  0.000000  0.0  0.000000  0.0  0.000000   
2498  0.033166  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.000000   
2499  0.089913  0.045185  0.0  0.000000  0.0  0.000000  0.0  0.000000   

            ag   ah  ...  zzf  zzi  zzk  zzl  zzo  zzs  zzt  zzu  zzy  zzz  
0     0.000000  0.0  ...  0.0  0.0  0.0  0.0  

In [49]:
DataFrame_Yelp_Train = pd.DataFrame(Vectorizer_Yelp_Train, columns = Vocabulary_Yelp_Train)
DataFrame_Yelp_Train = DataFrame_Yelp_Train.reindex(sorted(DataFrame_Yelp_Train.columns), axis=1)
print (DataFrame_Yelp_Train)

             a        a    aa        ab       ac        ad   ae        af  \
0     0.171214  0.048025  0.0  0.036015  0.00000  0.017194  0.0  0.014425   
1     0.140388  0.044025  0.0  0.012840  0.00000  0.036779  0.0  0.000000   
2     0.159648  0.054833  0.0  0.011994  0.04579  0.017178  0.0  0.000000   
3     0.190184  0.025403  0.0  0.022225  0.00000  0.000000  0.0  0.026706   
4     0.059365  0.035682  0.0  0.000000  0.00000  0.000000  0.0  0.000000   
...        ...       ...  ...       ...      ...       ...  ...       ...   
7495  0.103979  0.031249  0.0  0.000000  0.00000  0.000000  0.0  0.000000   
7496  0.157036  0.043564  0.0  0.000000  0.00000  0.000000  0.0  0.000000   
7497  0.043994  0.000000  0.0  0.000000  0.00000  0.000000  0.0  0.000000   
7498  0.061627  0.000000  0.0  0.000000  0.00000  0.000000  0.0  0.000000   
7499  0.170437  0.083817  0.0  0.016296  0.00000  0.000000  0.0  0.019582   

       ag   ah  ...  zzb  zzc  zze  zzi  zzl  zzn  zzo  zzu  zzy  zzz  
0  

In [50]:
DataFrame_Yelp_Test = pd.DataFrame(Vectorizer_Yelp_Test, columns = Vocabulary_Yelp_Train)
DataFrame_Yelp_Test = DataFrame_Yelp_Test.reindex(sorted(DataFrame_Yelp_Test.columns), axis=1)
print (DataFrame_Yelp_Test)

             a        a    aa        ab        ac        ad   ae        af  \
0     0.164301  0.026335  0.0  0.023041  0.000000  0.000000  0.0  0.013843   
1     0.078854  0.000000  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
2     0.090357  0.046551  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
3     0.120136  0.009628  0.0  0.000000  0.000000  0.000000  0.0  0.020244   
4     0.164085  0.024055  0.0  0.000000  0.000000  0.000000  0.0  0.010116   
...        ...       ...  ...       ...       ...       ...  ...       ...   
2495  0.120832  0.016139  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
2496  0.090460  0.036248  0.0  0.000000  0.000000  0.090846  0.0  0.000000   
2497  0.181204  0.019803  0.0  0.000000  0.000000  0.000000  0.0  0.041637   
2498  0.169448  0.040739  0.0  0.011881  0.015120  0.000000  0.0  0.014276   
2499  0.154315  0.037101  0.0  0.000000  0.041309  0.000000  0.0  0.000000   

            ag      ah  ...  zzb  zzc  zze  zzi  zzl  zzn  zzo 

## Akurasi dan Waktu Train Test Amazon dan Yelp

In [51]:
Waktu_Training = time()
RF_Classifier_Amazon = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
RF_Classifier_Amazon.fit(Vectorizer_Amazon_Train, Label_Amazon_Train)
print(f"\nWaktu Training Amazon: {round(time()-Waktu_Training, 3)}s")


Waktu Training Amazon: 52.493s


In [52]:
Waktu_Predict_Train = time()
Skor_Train_Amazon = RF_Classifier_Amazon.score(Vectorizer_Amazon_Train, Label_Amazon_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_Amazon = RF_Classifier_Amazon.score(Vectorizer_Amazon_Test, Label_Amazon_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest Train Amazon : {}".format(Skor_Train_Amazon))
print("Skor Random Forest Test Amazon : {}".format(Skor_Test_Amazon))

waktu prediksi (train): 5.67s
waktu prediksi (test): 1.932s

Skor Random Forest Train Amazon : 0.8589333333333333
Skor Random Forest Test Amazon : 0.8072


In [53]:
Waktu_Training = time()
RF_Classifier_Yelp = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
RF_Classifier_Yelp.fit(Vectorizer_Yelp_Train, Label_Yelp_Train)
print(f"\nWaktu Training Yelp: {round(time()-Waktu_Training, 3)}s")


Waktu Training Yelp: 55.101s


In [54]:
Waktu_Predict_Train = time()
Skor_Train_Yelp = RF_Classifier_Yelp.score(Vectorizer_Yelp_Train, Label_Yelp_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_Yelp = RF_Classifier_Yelp.score(Vectorizer_Yelp_Test, Label_Yelp_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest Train Yelp : {}".format(Skor_Train_Yelp))
print("Skor Random Forest Test Yelp : {}".format(Skor_Test_Yelp))


waktu prediksi (train): 5.819s
waktu prediksi (test): 1.997s

Skor Random Forest Train Yelp : 0.8698666666666667
Skor Random Forest Test Yelp : 0.8424


## Mencari Term-Term yang interseksi antara Amazon dan Yelp

In [55]:
Vocabulary_Intersection = []
iterator = 0
while iterator < len(Vocabulary_Yelp_Train) :
    if Vocabulary_Yelp_Train[iterator] in Vocabulary_Amazon_Train :
        Vocabulary_Intersection.append(Vocabulary_Yelp_Train[iterator])
    iterator = iterator + 1

In [56]:
def RecomputeIDF(WordDictAmazon, WordDictYelp, additional_term_value, Vocabulary_Intersection):
    idfDict = {}
    number_of_document_with_term_t_in_it = {}
    N = len(WordDictYelp) + len(WordDictAmazon)
    idfDict = dict.fromkeys(WordDictYelp[0].keys(), 0)
    for doc in WordDictYelp:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        if word in Vocabulary_Intersection :
            val = val + additional_term_value[word]
        idfDict[word] = np.log((1+N) / (1+val)) + 1
        
    return idfDict

In [57]:
New_IDF_Yelp = RecomputeIDF(WordDictionaryCount_Amazon_Train, WordDictionaryCount_Yelp_Train, number_of_document_with_term_t_Amazon_Train, Vocabulary_Intersection)

In [ ]:
New_TFIDF_nonScale_Yelp_Train = []
iterator = 0
while iterator < len(WordDictionaryCount_Yelp_Train):
    New_TFIDF_nonScale_Yelp_Train.append(computeTFIDFnonScale(CountFrequency_Sentimen_Yelp_Train[iterator], New_IDF_Yelp))
    iterator = iterator + 1

In [ ]:
New_TFIDF_Yelp_Train = []
iterator = 0
while iterator < len(WordDictionaryCount_Yelp_Train):
    tf_idf_list = list(New_TFIDF_nonScale_Yelp_Train[iterator].values())
    New_TFIDF_Yelp_Train.append(computeTFIDF(tf_idf_list))
    iterator = iterator + 1

In [ ]:
New_Vectorizer_Yelp_Train = []
iterator  = 0
while iterator < len(New_TFIDF_Yelp_Train):
    New_Vectorizer_Yelp_Train.append(list(New_TFIDF_Yelp_Train[iterator]))
    iterator = iterator + 1

In [ ]:
Waktu_Training = time()
New_RF_Classifier_Yelp = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
New_RF_Classifier_Yelp.fit(New_Vectorizer_Yelp_Train, Label_Yelp_Train)

In [ ]:
Waktu_Predict_Train = time()
New_Skor_Train_Yelp = New_RF_Classifier_Yelp.score(Vectorizer_Yelp_Train, Label_Yelp_Train)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")
print("Skor Random Forest Test Yelp : {}".format(New_Skor_Train_Yelp))

Waktu_Predict_Test = time()
New_Skor_Test_Yelp = New_RF_Classifier_Yelp.score(Vectorizer_Yelp_Test, Label_Yelp_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")
print("Skor Random Forest Test Yelp : {}".format(New_Skor_Test_Yelp))